In [1]:
# Initializing for the use of Earth Engine services
# In case of the first time use you should authenticate by pasting token in the box and then pressing enter
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

# The adapted v-HOTMAP algorithm (beta verison)
## The algorithm is implemented using Sentinel-2 images

The developed application allows to detect hotspots for a series of volcanoes from the list: Kilauea, Nyiragongo, Nyamuragira, Erta Ale, Villarrica, Yasur, Stromboli, Etna. Further the list of objects will be enlarged. The algorithm is an improved version of v-Hotmap algorithm.

In [2]:
import geemap
import io
import pandas as pd
from datetime import date
from ipyfilechooser import FileChooser
from ipyleaflet import (WidgetControl, Map, LayerGroup)
import colorama
from colorama import Fore
import ipywidgets as widgets
import os
import requests
import numpy as np

In [3]:
#df with predefined volcanoes and their coordinates
# Add here new volcanoes with their coordinates if you want
# Volcano database: https://volcano.si.edu/
volcano_list = ['Kilauea', 'Nyiragongo', 'Nyamuragira', 'Erta Ale', 'Villarrica', 'Yasur', 'Stromboli', 'Etna']
data_volc = {'volcano': volcano_list,
        'x_coords': [-155.287, 29.25, 29.2, 40.67, -71.93, 169.447, 15.213, 14.999], 
        'y_coords': [19.421, -1.52, -1.408, 13.6, -39.42, -19.532, 38.789, 37.748] }
volcanoes = pd.DataFrame(data_volc)

In [4]:
#map visualization
# turn on the map and add layer control to list all present layers on a map
m = geemap.Map()
m.add_basemap('TERRAIN')
m.addLayerControl()
m

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

***
* __Select the volcano and buffer distance:__

In [5]:
#widgets for volcano dropdown list and buffer distance selection
style = {'description_width': 'initial'}

# Output widget to indicate the period of processing, added on the map in right bottom part
output_widget = widgets.Output()
output_control = WidgetControl(widget=output_widget, position='bottomright')
m.add_control(output_control)

# Widget with volcanoes, add to the list in chunk 3 new volcanoes
volcano_widget = widgets.Dropdown(description='Select Volcano:',
                                  options=volcano_list, style=style)
# Widget to indicate bufet and final layout and displaying
buffer_widget = widgets.Text(description='Buffer distance:', max_height=20,  style=style)
volc_widgets = widgets.HBox([volcano_widget, buffer_widget])
display(volc_widgets)

* __Select the date interval:__

In [6]:
#Widgets to select the date period of interest
start_day = widgets.DatePicker(description='Start date: ', disabled=False)
end_day = widgets.DatePicker(description='End date: ', disabled=False)
widget_dates = widgets.HBox([start_day, end_day])
#Displaying widgets
display(widget_dates)

* __Select the visualization parameters:__

In [19]:
#widgets for setting visualization parameters
widget_color = widgets.ColorPicker(concise=False, description='Hotspot color:', value= 'red', style=style) # Selection of color to visualize hotposts
visbands_widget = widgets.Dropdown(description='Select band combination:',
                                  options=['Natural Colors', 'NIR SWIR Colors', 'False Color', 'PAN'], style=style) # Selection of visualization
sentinel_stretch = widgets.BoundedIntText(description='Stretching max value:', max_height=20, value = 3000, min = 0, max = 10000, step = 100,  style=style) # Max stretching value for the image visualization

# Widgets layout and displaying
widget_wis = widgets.HBox([widget_color, visbands_widget, sentinel_stretch]) 
display(widget_wis)

***
* __Click submit to visualize the hotspots (or clear to close the layers and make a new selection):__

In [8]:
# Widgets for submission and removing all layers
# Buttons to submit and clear the map by removing layers
submit = widgets.Button(description='Submit', button_style='primary', tooltip='Click to submit', style=style)
clear = widgets.Button(description='Clear', button_style='warning', tooltip='Click to clear', style=style)
# Output widget to confirm succesfull sumbission or print errors
output_widget_submit = widgets.Output()
# Layout and displaying
widget_buttons = widgets.HBox([submit, clear])
full_widget = widgets.VBox([widget_buttons, output_widget_submit])
display(full_widget)

***
* __Select the folder to download hotspots layers or a final table:__

In [9]:
#Widgets to export data
# Buttons to export tif mask files and to export in a table format
export = widgets.Button(description='Export data', button_style='info', tooltip='Click to export tif', style=style)
export_table = widgets.Button(description='Export table', button_style='info', tooltip='Click to export table', style=style)
# Output widget to confirm the export and display thar export is in progress, or to show errors
output_widget_export = widgets.Output()
# Widget to choose location for a file to save
fc = FileChooser('')
fc.show_only_dirs = True
# Layout and diaplaying
export_widget = widgets.VBox([fc, widgets.HBox([export, export_table]), output_widget_export])
display(export_widget)

In [10]:
# Event handler when the button submit is clicked
def on_submit_clicked(b):
    # while processing display "processing" in the output widget
    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        
        try:
            # define band visualizition depending on the selected user choice
            bands = ['B12', 'B8A', 'B5']
            if visbands_widget.value == 'Natural Colors':
                bands = ['B4','B3','B2']
            elif visbands_widget.value == 'NIR SWIR Colors':
                bands = ['B12', 'B8A', 'B5']
            elif visbands_widget.value == 'False Color':
                bands = ['B8', 'B4', 'B3']
            elif visbands_widget.value == 'PAN':
                bands = ['B8', 'B8', 'B8']
            
            # global parameters to be accesed outside the function
            global visParams
            visParams = {'bands': ['B1'], 'palette': [widget_color.value]} # visualization of hotspots, take the value from widget
            global visParams_sentinel2
            visParams_sentinel2 = {"min": 0, "max": int(sentinel_stretch.value), "bands": bands} # parameters to visualize the image, value from the widget
            global start
            start = ee.Date(start_day.value.strftime('%Y-%m-%d')) # date of the start, access widget and transform
            global finish
            finish = ee.Date(end_day.value.strftime('%Y-%m-%d')) # date of the start, access widget and transform
            global volcano
            volcano = volcano_widget.value # volcano of interest, access the widget
            global buffer_dist
            buffer_dist = buffer_widget.value # buffer distance, access the widget

            volcano_df = volcanoes[volcanoes['volcano'].str.contains(volcano)] # retrieve the row with volcano name
            # Access the coordinates of the volcano
            x_coords = volcano_df.iat[0, 1] 
            y_coords = volcano_df.iat[0, 2]
            
            # make a volcano point and buffer, bounding box for buffer to obtain squre around the point
            global point_volcano
            point_volcano = ee.Geometry.Point([x_coords, y_coords])
            buffer = point_volcano.buffer(int(buffer_dist))
            bbox = buffer.bounds()
            global roi
            roi = bbox
            m.addLayer(bbox, {'color': 'blue'}, 'Area of interest') # Add layer of AOI
            
            # Zoom and center on the AOI
            zoom = 14
            m.setCenter(x_coords, y_coords, zoom)
            
            # Retrieve the collection and fid its size
            s2_coll = collection_retr(start, finish, roi)
            imagelist= s2_coll.toList(s2_coll.size()) #find the size of the collection
            list_length_raw = ee.String(imagelist.length())
            list_length_str = list_length_raw.getInfo()
            image_number_auto = list_length_str-1
            
            #Get sentinel series of images for the period of interest (global to be accesed from hotmap fucntion)
            global hotmap_ser
            sentinel_ser = sentinel_series(imagelist, image_number_auto)
            # Get hotmap masks for all images in the sentinel series
            hotmap_ser = hotmap_series(sentinel_ser)
        
            output_widget.clear_output()
            
        #catch errors when the date range is not defined correctly
        except Exception as e:
            with output_widget_submit:
                output_widget_submit.clear_output()
                print(Fore.RED + 'Error occured! Check the dates range: no images found for the inidcated period')
            
submit.on_click(on_submit_clicked)

In [20]:
#Event handler to remove all layers
def on_clear_clicked(button):
    with output_widget:
        output_widget.clear_output()
        print('Clearing...')
        try:
            # Set all layers to default values
            start_day.value = None
            end_day.value = None
            widget_color.value = 'red'
            sentinel_stretch.value = '3000'
            volcano_widget.value = 'Kilauea'
            buffer_widget.value = ''
            
            #clear map and rest a terrain basemap
            m.clear_layers()
            m.add_basemap('TERRAIN')
            
            #clear output widgets
            output_widget_submit.clear_output()
            output_widget.clear_output()
            output_widget_export.clear_output()
        except Exception as e:
            with output_widget_submit:
                print(Fore.RED + 'Error occured! Cannot remove the layers')
                print(Fore.RED + e)

clear.on_click(on_clear_clicked)

In [12]:
#Event handler to export the data in tif format (hotmap layers)
def export_hotmap_clicked(b):
    # print in an output widget (as it may take a while)
    with output_widget_export:
        output_widget_export.clear_output()
        print('Exporting...')
        try:
            # access the stored series of images that was visualized previosly
            # save every image from a list in a loop
            for image in hotmap_ser:
                image_date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
                export_name = 'S2-v-HOTMAP_' + volcano + '_' + image_date.getInfo() + '.tif'
                
                image_to_export = image.toInt() # to integer 1 and 0 mask
                #download url for images (works with small under 24 mb images), to avoid saving on google drive
                url = image_to_export.getDownloadURL( {
                    'bands' : ['B1'],
                    'name' : export_name,
                    'region' : roi,
                    'scale' : 20,
                    'format': 'GEO_TIFF'
                })
                response = requests.get(url) # access the response from url
                
                # access the directory to save file
                global directory
                directory = fc.value
                dir_file = directory
                dir_file.replace(r'\\', '/') # adjust for python backslahes

                complete_name = os.path.join(dir_file, export_name) # full name of the image with path
                # save the image
                with open(complete_name, 'wb') as fd:
                    fd.write(response.content)
                # Notify about the succesfull operation    
                output_widget_export.clear_output()
                print('Exported succesfully!')
        # Handle most common errors and all other errors
        except NameError as namerr:
            output_widget_export.clear_output()
            print(Fore.RED + 'Error: click submit before. No data to export found')
        except AttributeError as attrer:
            output_widget_export.clear_output()
            print(Fore.RED + 'Error: the path to save the file is not inidcated')
        except Error as e:
            output_widget_export.clear_output()
            print(Fore.RED + ' Unknown error occured. Try to clear and resubmit your request')
            
export.on_click(export_hotmap_clicked)       

In [13]:
#Event handler to export the data in table format
def table_export(b):
    # print exporting with output widget (as it could take a while)
    with output_widget_export:
        output_widget_export.clear_output()
        print('Exporting...')
        try:
            # create to lists to save dates and counted values of mask pixels
            List_dates = []
            List_values = []
            
            # lop through the series of images
            for image in hotmap_ser:
                #access hotmap series and select dates
                image_date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
                List_dates.append(image_date)
                
                #getting image url to save it in numpy array format
                image_to_export = image.toInt()
                url = image_to_export.getDownloadURL( {
                    'bands' : ['B1'],
                    'name' : 'Hotmap',
                    'region' : roi,
                    'scale' : 20,
                    'format': 'NPY'
                })
                
                #tranformation to "readable" numpy format
                response = requests.get(url)
                numpy_array_raster = np.load(io.BytesIO(response.content))
                numpy_array_raster = numpy_array_raster.tolist()
                
                #loop through array to count the number of hotmap occurences
                sum_positive = 0
                for elem in numpy_array_raster:
                    for el in elem:
                        if el[0] == 1:
                            sum_positive += 1

                List_values.append(sum_positive)
                
            #combine the final dataframe for the output from date list and values list
            final_stats = pd.DataFrame( {
                'Date': List_dates,
                'Pixel_number': List_values
            })
            
            #save the file
            global directory
            directory = fc.value
            dir_file = directory
            dir_file.replace(r'\\', '/')

            export_name = 'S2-HOTMAP_' + volcano + '.csv' #name for the file

            complete_name = os.path.join(dir_file, export_name) # full path
            final_stats.to_csv(complete_name) # save as csv file
            
            output_widget_export.clear_output()
            print('Exported succesfully!')
        
        # handle possible exceptions
        except NameError as namerr:
            output_widget_export.clear_output()
            print(Fore.RED + 'Error: click submit before. No data to export found')
        except AttributeError as attrer:
            output_widget_export.clear_output()
            print(Fore.RED + 'Error: the path to save the file is not inidcated')
        except Error as e:
            output_widget_export.clear_output()
            print(Fore.RED + ' Unknown error occured. Try to clear and resubmit your request')

export_table.on_click(table_export)

In [14]:
# Retrieve the collection and filter it by bounds and AOI
def collection_retr(start, finish, roi):
    collection = "COPERNICUS/S2" 
    s2_coll = (ee.ImageCollection(collection)
        .filterDate(start, finish)
        .filterBounds(point_volcano))
    return s2_coll

In [15]:
def HOTMAP(image):
  # Variable selection (2 SWIR and 1 NIR(B8A))
  alpha1 = image.expression('B12/B11', {'B12': image.select('B12'), 'B11' : image.select('B11')})
  alpha2 = image.expression('B12/B8A', {'B12': image.select('B12'), 'B8A' : image.select('B8A')})

  beta1 = image.expression('B11/B8A', {'B11': image.select('B11'), 'B8A' : image.select('B8A')})

  B12 = image.select('B12').divide(10000) # Reflectance is scaled by 10000
  B11 = image.select('B11').divide(10000)
  B8  = image.select('B8A').divide(10000)
  # Select the cloud band and rescale from 60 to 20
  Cloud = image.select('QA60')
  proj = Cloud.projection().getInfo()
  crs = proj['crs']
  Clouds = Cloud.resample().reproject(**{'crs' : crs, 'scale' : 20.0})

  #Preparing the Alpha parameter

  alpha_A_raw = image.where(alpha1.gte(1.4),1) # Make binary map for each image (gte == greater than or equal; neq == not equal)
  alpha_A = alpha_A_raw.where(alpha_A_raw.neq(1), 0) # second parameter is the new value

  alpha_B_raw = image.where(alpha2.gte(2.0),1)
  alpha_B = alpha_B_raw.where(alpha_B_raw.neq(1),0)

  alpha_C_raw = image.where(B12.gte(0.6),1)
  alpha_C = alpha_C_raw.where(alpha_C_raw.neq(1),0)


  Alpha_total = image.expression('alpha + alphaa + alphaaa', { 'alpha' : alpha_A, 'alphaa': alpha_B, 'alphaaa': alpha_C}) # sum the binary images

  Alpha_raw = Alpha_total.where(Alpha_total.eq(3),1) # only those for which all 3 are correct 
  Alpha = Alpha_raw.where(Alpha_total.neq(3),0) # Use Alpha_total here

  # preparing the Beta parameter
  s = image.where(B11.gt(1),1) # Saturation values above 1 for both bands
  ss= s.where(B12.gt(1),1)
  S= ss.where(ss.neq(1),0)

  beta_A_raw = image.where(beta1.gte(2),1)
  beta_A = beta_A_raw.where(beta_A_raw.neq(1),0)

  beta_B_raw = image.where(B11.gte(0.5),1)
  beta_B = beta_B_raw.where(beta_B_raw.neq(1),0)

  beta_AB = image.expression('beta+betaa', { 'beta': beta_A, 'betaa': beta_B,}) # sum the binary images

  beta_C_raw = beta_AB.where(beta_AB.eq(2),1)
  beta_C = beta_C_raw.where(beta_AB.neq(2),0)

  beta_D_raw = beta_C.where(beta_C.eq(1),1)
  beta_D = beta_D_raw.where(S.eq(1),1)

  Beta = beta_D.where(beta_D.neq(1),0)

  # Combining both parameters

  Hot_pixels_raw = image.where(Beta.eq(0),0)
  Hot_pixels_raw2 = Hot_pixels_raw.where(Alpha.eq(0),0)
  Hot_pixels_raw3 = Hot_pixels_raw2.where(Beta.eq(1),1)
  Hot_pixels_raw4 = Hot_pixels_raw3.where(Alpha.eq(1),1)
  Hot_pixels = Hot_pixels_raw4.where(Clouds.eq(1<<10),0) # Setting all rescaled cloud pixels (value of 1 in the 10th bit) to zero

  # First clustering
  
  Cluster = Hot_pixels.connectedComponents(connectedness = ee.Kernel.square(1), maxSize = 128) # Find all the clusters in the image (square(1) == 8 pixel search area)
  Cluster = Cluster.select(['labels']) # Each cluster gets a unique label (int)
  Alpha = Alpha.addBands(Cluster.select(['labels'])) # for each Alpha pixel, the corresponding label is added (0 value (beta) pixels also included)
  True_clusters = Alpha.reduceConnectedComponents(reducer = ee.Reducer.mean(), labelBand = 'labels') # The mean Alpha value of each cluster is calculated, if 1 alpha is present => retain cluster
  Hotspot = Hot_pixels.where(True_clusters.gt(0),1) # Every cluster with mean Alpha value above 0 is retained
  Hotspot = Hotspot.where(True_clusters.eq(0),0)
  
  # Cloud clustering
  Cloud_seethrough_raw = image.where(Clouds.eq(1<<10),1) # Cloud images are included
  Cloud_seethrough = Cloud_seethrough_raw.where(Cloud_seethrough_raw.neq(1),0)

  # new Alpha parameters
  gamma_A_raw = image.where(B12.gte(0.9),1)
  gamma_A = gamma_A_raw.where(gamma_A_raw.neq(1),0)

  gamma_B_raw = image.where(alpha1.gte(1.4),1)
  gamma_B = gamma_B_raw.where(gamma_B_raw.neq(1),0)

  gamma_C_raw = image.where(alpha2.gte(1.65),1)
  gamma_C = gamma_C_raw.where(gamma_C_raw.neq(1),0)

  
  gamma_ABC = image.expression('gamma + gammaa + gammaaa + gammaaaa', { 'gamma': Cloud_seethrough, 'gammaa': gamma_A, 'gammaaa' : gamma_B, 'gammaaaa':gamma_C})

  gamma_raw = image.where(gamma_ABC.eq(4),1)
  gamma = gamma_raw.where(gamma_ABC.neq(4),0)
  H= image.where(Beta.eq(0),0)
  H2 = H.where(gamma.eq(0),0)
  # Original Beta is included to prevent loss of clear image Beta-pixels
  H3 = H2.where(Beta.eq(1),1)
  H4 = H3.where(gamma.eq(1),1)
  # Second clustering
  Cluster = H4.connectedComponents(connectedness = ee.Kernel.square(1), maxSize = 128) # Similar approach
  Cluster = Cluster.select(['labels'])
  gamma = gamma.addBands(Cluster.select(['labels']))
  True_clusters = gamma.reduceConnectedComponents(reducer = ee.Reducer.mean(), labelBand = 'labels')
  Hotspot2 = H4.where(True_clusters.gt(0),1)
  Hotspot2 = Hotspot2.where(True_clusters.eq(0),0)

  final = Hotspot.where(Hotspot2.gt(0),1)
  clipped = final.clip(roi)
  Mask = clipped.selfMask()

  return ee.Image(Mask) 

In [16]:
#retrival of hotmap series used in further analysis
def hotmap_series(sentinel_series):
    hotmap_list = []
    List_dates = []
    # for every image in list of images to create a hotmap mask and visualize it on a map
    for image in sentinel_series:
        image1_hotmap = HOTMAP(image)
        image_date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        hotmap_list.append(image1_hotmap)
        m.addLayer(image1_hotmap, visParams, "v-Hotmap; Date: " + image_date.getInfo()) # visualizing
        
    return hotmap_list     

In [17]:
#visualization of sentinel images and return of sentinel series without double-date images
def sentinel_series(imagelist, image_number_auto):
    sentinel_list = []
    List_date = []
    i = 0
    for x in range(0, image_number_auto): # loop theough the list
        imageID1 = imagelist.get(x)
        image1 = ee.Image(imageID1).clip(roi)
        image_date2 = ee.Date(image1.get('system:time_start')).format('YYYY-MM-dd')
        
        if image_date2.getInfo() in List_date: # to avoid double per one date
            continue
        List_date.append(image_date2.getInfo())
        sentinel_list.append(image1) # append a list with all images for period of interest
        m.addLayer(image1, visParams_sentinel2, visbands_widget.value + "; Date: " + image_date2.getInfo()) # add layer on the map
            
    return sentinel_list   